# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!


### Text Classification
> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)

---
#### 

### 0. library

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 340 kB/s 
     |████████████████████████████████| 453 kB 66.8 MB/s 


In [ ]:
## import sklearn
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.backend import clear_session
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


### 1. 데이터 가져오기

In [ ]:
# 2. data preprocessing에서 저장한 결과를 불러오고 shape을 확인합니다.
import pickle

In [ ]:
# sequential
with open("/content/drive/MyDrive/Colab Notebooks/KT_mini_4/x_train_seq.pickle", 'rb') as f:
    x_train_seq = pickle.load(f)
    
with open("/content/drive/MyDrive/Colab Notebooks/KT_mini_4/x_valid_seq.pickle", 'rb') as f:
    x_valid_seq = pickle.load(f)

In [ ]:
# y_train
with open("/content/drive/MyDrive/Colab Notebooks/KT_mini_4/y_train.pickle", 'rb') as f:
    y_train = pickle.load(f)
    
# y_valid
with open("/content/drive/MyDrive/Colab Notebooks/KT_mini_4/y_valid.pickle", 'rb') as f:
    y_valid = pickle.load(f)

### 3. Deep Learning(Sequence)

In [ ]:
x_train_seq.shape, x_valid_seq.shape

((15066, 23), (5023, 23))

In [ ]:
# sequence 형태로 vectorize한 데이터를 딥러닝으로 학습합니다.
# Tip : 처음에는 embedding이 필요합니다

vocab_size = 27701 # 사전 (토크나이저) 의 개수
embedding_dim = 50
hidden_units = 256

clear_session

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dense(256, activation='relu') )
model.add(Dense(512, activation='relu') )
model.add(Dense(1024, activation='relu') )
model.add(Dense(1, activation='softmax') )

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1385050   
                                                                 
 dense (Dense)               (None, None, 256)         13056     
                                                                 
 dense_1 (Dense)             (None, None, 512)         131584    
                                                                 
 dense_2 (Dense)             (None, None, 1024)        525312    
                                                                 
 dense_3 (Dense)             (None, None, 1)           1025      
                                                                 
Total params: 2,056,027
Trainable params: 2,056,027
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train_seq, y_train, epochs=15, callbacks=[es, mc], batch_size=16, validation_split=0.25)

Epoch 1/15
699/707 [============================>.] - ETA: 0s - loss: 5.4771 - acc: 0.6408
Epoch 1: val_acc improved from -inf to 0.62304, saving model to best_model.h5
707/707 [==============================] - 9s 6ms/step - loss: 5.4646 - acc: 0.6416 - val_loss: 5.7483 - val_acc: 0.6230
Epoch 2/15
703/707 [============================>.] - ETA: 0s - loss: 5.4677 - acc: 0.6414
Epoch 2: val_acc did not improve from 0.62304
707/707 [==============================] - 4s 6ms/step - loss: 5.4646 - acc: 0.6416 - val_loss: 5.7483 - val_acc: 0.6230
Epoch 3/15
699/707 [============================>.] - ETA: 0s - loss: 5.4526 - acc: 0.6424
Epoch 3: val_acc did not improve from 0.62304
707/707 [==============================] - 4s 6ms/step - loss: 5.4646 - acc: 0.6416 - val_loss: 5.7483 - val_acc: 0.6230
Epoch 4/15
704/707 [============================>.] - ETA: 0s - loss: 5.4680 - acc: 0.6414
Epoch 4: val_acc did not improve from 0.62304
707/707 [==============================] - 4s 6ms/step - 

#### 3-2. LSTM

In [ ]:
x_train_seq.shape, x_valid_seq.shape

((15066, 23), (5023, 23))

In [ ]:
y_train.shape, y_valid.shape

((15066,), (5023,))

In [ ]:
vocab_size = 27701 # 사전의 개수 = 토크나이저의 개수
embedding_dim = 100
hidden_units = 128

clear_session

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 100)         2770100   
                                                                 
 lstm_3 (LSTM)               (None, 128)               117248    
                                                                 
 dense_5 (Dense)             (None, 1024)              132096    
                                                                 
 dense_6 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 3,020,469
Trainable params: 3,020,469
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train_seq, y_train, epochs=15, callbacks=[es, mc], batch_size=8, validation_split=0.25)

Epoch 1/15
1407/1413 [============================>.] - ETA: 0s - loss: 0.3579 - acc: 0.8210
Epoch 1: val_acc improved from -inf to 0.87364, saving model to best_model.h5
1413/1413 [==============================] - 11s 6ms/step - loss: 0.3579 - acc: 0.8210 - val_loss: 0.2808 - val_acc: 0.8736
Epoch 2/15
1412/1413 [============================>.] - ETA: 0s - loss: 0.0563 - acc: 0.9805
Epoch 2: val_acc did not improve from 0.87364
1413/1413 [==============================] - 9s 6ms/step - loss: 0.0563 - acc: 0.9805 - val_loss: 0.4081 - val_acc: 0.8675
Epoch 3/15
1409/1413 [============================>.] - ETA: 0s - loss: 0.0277 - acc: 0.9890
Epoch 3: val_acc did not improve from 0.87364
1413/1413 [==============================] - 8s 6ms/step - loss: 0.0276 - acc: 0.9890 - val_loss: 0.3675 - val_acc: 0.8705
Epoch 4/15
1407/1413 [============================>.] - ETA: 0s - loss: 0.0224 - acc: 0.9904
Epoch 4: val_acc did not improve from 0.87364
1413/1413 [==============================]

In [ ]:
y_predict = model.predict(x_valid_seq)

In [ ]:
y_predict

array([[1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       ...,
       [1.0000000e+00],
       [5.4129737e-07],
       [1.0000000e+00]], dtype=float32)

In [ ]:
import numpy as np

y_pred = np.where(y_predict < 0.5, 0, 1)

In [ ]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.92      0.82      1803
           1       0.95      0.82      0.88      3220

    accuracy                           0.86      5023
   macro avg       0.84      0.87      0.85      5023
weighted avg       0.87      0.86      0.86      5023



### 3-3. Transformer

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
vocab_size = 27701
maxlen = 23 

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
embed_dim = 100  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 23)]              0         
                                                                 
 token_and_position_embeddin  (None, 23, 100)          2772400   
 g_2 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_2 (Transf  (None, 23, 100)          106928    
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_6 (Dropout)         (None, 100)               0   

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(x_train_seq, y_train, epochs=15, callbacks=[es, mc], batch_size=8, validation_split=0.25)

Epoch 1/15
1407/1413 [============================>.] - ETA: 0s - loss: 0.0301 - accuracy: 0.6412

1413/1413 [==============================] - 13s 8ms/step - loss: 0.0300 - accuracy: 0.6416 - val_loss: 0.2047 - val_accuracy: 0.6230
Epoch 2/15
1412/1413 [============================>.] - ETA: 0s - loss: 0.0234 - accuracy: 0.6416

1413/1413 [==============================] - 11s 8ms/step - loss: 0.0234 - accuracy: 0.6416 - val_loss: 0.2250 - val_accuracy: 0.6230
Epoch 3/15
1408/1413 [============================>.] - ETA: 0s - loss: 0.0183 - accuracy: 0.6413

1413/1413 [==============================] - 11s 8ms/step - loss: 0.0184 - accuracy: 0.6416 - val_loss: 0.2567 - val_accuracy: 0.6230
Epoch 4/15
1406/1413 [============================>.] - ETA: 0s - loss: 0.0254 - accuracy: 0.6413

1413/1413 [==============================] - 11s 8ms/step - loss: 0.0255 - accuracy: 0.6416 - val_loss: 0.2935 - val_accuracy: 0.6230
Epoch 5/15
1411/1413 [============================>.] - ETA: 0s - loss: 0.0215 - accuracy: 0.6417

1413/1413 [==============================] - 11s 8ms/step - loss: 0.0214 - accuracy: 0.6416 - val_loss: 0.2650 - val_accuracy: 0.6230
Epoch 6/15
1411/1413 [============================>.] - ETA: 0s - loss: 0.0187 - accuracy: 0.6418

1413/1413 [==============================] - 11s 8ms/step - loss: 0.0187 - accuracy: 0.6416 - val_loss: 0.2830 - val_accuracy: 0.6230
Epoch 6: early stopping


In [ ]:
y_predict = model.predict(x_valid_seq)
y_pred = np.where(y_predict < 0.5, 0, 1)
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1803
           1       0.64      1.00      0.78      3220

    accuracy                           0.64      5023
   macro avg       0.32      0.50      0.39      5023
weighted avg       0.41      0.64      0.50      5023



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
